# Linear Regression

- **Task** - Regression: learn a function from one or more real value inputs to a real value output
- **Model/hypothesis** - a linear function

In [1]:
var _ = require('lodash');
var loader = require('csv-load-sync');
var d3 = require('d3');
var { JSDOM } = require('jsdom');
var {d3svg, d3bar, d3plot, d3fit, d3decision, argMax} = require('./utilities.js');

## Sample Data

In [2]:
var iris = loader('iris.csv')
iris[0]

{ Sepal_Length: '5.1',
  Sepal_Width: '3.5',
  Petal_Length: '1.4',
  Petal_Width: '0.2',
  Species: 'setosa' }

In [3]:
var petalWidthHeight = iris.map(o => { return {x: o['Petal_Width'], y: o['Petal_Length']}; });
d3plot(petalWidthHeight, 'width', 'length')

## Univariate regression

Predict petal length given the petal width.

- **Inputs (Features)** - petal width
- **Output (Target)** - petal length

## Hypothesis

- **Linear**: $f(x) = \alpha x + \beta$
- **Parameters (Weights)**: p = $[\alpha, \beta]$

In [4]:
var hypothesis = (x, p) => p[0] * x + p[1];

## Cost function

> Also called error or loss function.

In [5]:
d3fit(petalWidthHeight, 'width', 'length', x => hypothesis(x, [1,1]))

**Mean Squared Error (MSE)**

In [6]:
function calcCost(data, f) {
  var sum = 0;
  for (var pt of data) {
    sum += Math.pow(f(pt.x) - pt.y, 2);
  }
  return sum / data.length;
}

In [7]:
calcCost(petalWidthHeight, x => hypothesis(x, [1,1]))

3.5281333333333333

In [8]:
d3fit(petalWidthHeight, 'width', 'length', x => hypothesis(x, [2,2]))

In [9]:
calcCost(petalWidthHeight, x => hypothesis(x, [2,2]))

0.6666

## Training

"Learn" a function from petal width to petal length by finding the "best" value of the parameters $[\alpha, \beta]$. 

![gradient descent](https://ml-cheatsheet.readthedocs.io/en/latest/_images/gradient_descent_demystified.png)

In [10]:
function learnStep(data, model, initial, alpha) {
    var deltaSum = [0,0];

    var guess = initial;
    for (var pt of data) {
        var out  = model(pt.x, guess);
        deltaSum[1] += out - pt.y;
        deltaSum[0] += (out - pt.y) * pt.x;
    }

    var k = alpha / data.length;
    return [guess[0] - k * deltaSum[0], guess[1] - k * deltaSum[1]];
}
function learn(data, model, guess, alpha, steps) {
    for (var i = 0; i < steps; i++)
        guess = learnStep(data, hypothesis, guess, alpha);
    return guess;
}

In [11]:
var batch = 100;
var guess = [1,1];
var learnCurve = [];
for (var i = 0; i < 15; i++) {
    var cost = calcCost(petalWidthHeight, x => hypothesis(x, guess));
    learnCurve.push(cost);
    console.log(guess, cost);
    guess = learn(petalWidthHeight, hypothesis, guess, 0.001, batch);
}
d3plot(learnCurve.map((c,i) => {return {x:i*batch,y:c};}), 'iterations', 'cost')

[ 1, 1 ] 3.5281333333333333
[ 1.2259327816552261, 1.1348249953238987 ] 2.136515863800172
[ 1.3979882368754635, 1.2342554805286354 ] 1.342653608213844
[ 1.5293502140800959, 1.3070203571151398 ] 0.8893202198844545
[ 1.6299694330471521, 1.3597065205758827 ] 0.6299979509209915
[ 1.707356501318104, 1.3972842761618465 ] 0.4812286113823229
[ 1.7671804179497292, 1.4235038790459402 ] 0.39547273188975895
[ 1.8137202751255341, 1.4411948097999605 ] 0.34565017348124016
[ 1.8502061655933857, 1.4524916425024914 ] 0.31633447536738324
[ 1.8790764717779447, 1.4590045110164227 ] 0.2987371636148745
[ 1.902172046660965, 1.4619477625340358 ] 0.2878508169593567
[ 1.9208827657591385, 1.4622370543217598 ] 0.280821758094384
[ 1.9362581327230735, 1.4605626339975761 ] 0.27602313820110463
[ 1.9490907555644594, 1.4574446451069487 ] 0.2725269646933036
[ 1.9599793478624652, 1.4532748668810245 ] 0.2698033666153116


## Trained Model

The hypothesis + learned parameters → trained model.

In [12]:
var predict = (x) => hypothesis(x, guess);
d3fit(petalWidthHeight, 'width', 'length', predict)